In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
import zipfile
import warnings
from datetime import datetime


##### [RQ7] Estimating probabilities is a core skill for a data scientist: show us your best!

- Estimate the probability that a book has over 30% of the ratings above 4.
- Estimate the probability that an author publishes a new book within two years from its last work.
- In the file list.json, you will find a peculiar list named "The Worst Books of All Time." Estimate the probability of a book being included in this list, knowing it has more than 700 pages.
- Are the events X=’Being Included in The Worst Books of All Time list’ and Y=’Having more than 700 pages’ independent? Explain how you have obtained your answer.

In [15]:
# load data
# filename = "lighter_authors"
filename = "lighter_books"
books_head = None
df = pd.DataFrame()
flag = True

# load config
with open('config.json') as cfg:
    cfg = json.load(cfg)
    path = os.path.join(cfg["datadir"], filename + '.' + "json")
    print(path)
    processed_rows = 0

    chunks = pd.read_json(path, lines=True, chunksize = 10**3, nrows = 10**4, dtype={'num_pages':'int64'})
    for chunk in chunks:
        if flag:
            df = chunk[['author_id','original_publication_date']] # ['author_id','original_publication_date']
            flag = False
        else:
            df = pd.concat([df, chunk[['author_id','original_publication_date']]  ], axis=0)
            # df = pd.concat([df, chunk['author_id','original_publication_date']], axis=0)
        # full_df = pd.concat( , axis = 0)
        # books_head = chunk[['author_id','original_publication_date']]
        # break
df


F:/adm/lighter_books.json


,author_id,original_publication_date
0,1077326,2003-06-21
1,1077326,1997-06-26
2,1077326,1998-07-02
3,1077326,1999-07-08
4,1077326,2000-07-08
...,...,...
10994,10263,1997
10995,10263,1995
10996,10263,2004-01-01
10997,10263,


In [110]:
def myfun2(*args, **kwargs):
    return_value = None
    res = list(*args, **kwargs)
    print(res)
    res = [ myStringToDate(entry, '%Y-%m-%d') for entry in res ]
    res = [ entry for entry in res if entry is not None ]
    res = [ myDateToSeconds(res[0], entry) for entry in res]
    res = sorted(set(res)) # first eliminate duplicates, then sort
    return_value = []
    for i in range(len(res)-1):
        return_value.append(res[i+1]-res[i])
    if len(return_value)>0:
        return_value = sum(return_value)/len(return_value) <= 2*60*60*24*365 # returns true if the author is expected to publish a new book within (<=) 2 years
    return return_value

def myStringToDate(date_string, date_format):
    res = None
    try:
        res = datetime.strptime(date_string, date_format)
    except ValueError as e:
        print(e)
    return res

def myDateToSeconds(initial_date, current_date):
    res = None
    try:
        res = current_date - initial_date
        res = res.total_seconds() 
    except Exception as e:
        print(e)
    return res


In [111]:
# df.groupby('author_id')['original_publication_date'].apply(list)
df_new = pd.DataFrame(df.groupby('author_id')['original_publication_date'].apply(myfun2))
df_new

['1979-10-12', '1996-01-17', '1996-01-17', '1979-10-12', '1979-10-12', '2005-05-23', '1996-01-17', '2005-06-20', '1988-10-10', '1987-05-15', '2002-05-28', '1992-10-12', '1982-12-29', '1988-10-10', '1988-10-10', '1987-05-15', '2002-05-28', '2002-05-28', '2002-05-28', '2002-05-28', '2002-05-28', '2002-05-28', '1990', '1987-05-15', '1979-10-12', '1982-12-29', '1980-10-12', '1990-02-01', '1984-10-12', '1979', '1999-07-05', '1984-10-12', '1984-10-12', '1980-10-12', '1987', '1979-10-12', '1985-10', '1992-10-12', '1985-01-01', '1979']
time data '1990' does not match format '%Y-%m-%d'
time data '1979' does not match format '%Y-%m-%d'
time data '1987' does not match format '%Y-%m-%d'
time data '1985-10' does not match format '%Y-%m-%d'
time data '1979' does not match format '%Y-%m-%d'
['2003-05-06', '2002', '1984', '2000-06-18', '1999-05-12', '1989-08-01', '1991', '1995-09-07', '1990-07-17', '2003-05-06', '2003-05-06', '2003-05-06', '2003-05-06', '1998-05-05', '1998-05-05', '1998-05-05', '1998-

,original_publication_date
author_id,
4,True
7,True
10,False
12,[]
16,False
...,...
20148549,[]
20378624,[]
20411801,[]


In [133]:
# df_new.groupby('original_publication_date')['original_publication_date'].apply(list)
# df_new[]
def listlen(*args,**kwargs):
    return len(list(*args))

results = df_new[ (df_new['original_publication_date'] == True) | (df_new['original_publication_date'] == False)].groupby('original_publication_date')['original_publication_date'].apply(listlen)
# df_new[ df_new['original_publication_date'] == True]
# pd.DataFrame(results)
results.loc[True]/(results.loc[True]+results.loc[False]) # probability


0.24906367041198502